In [8]:
from googleapiclient.discovery import build
import pandas as pd

In [9]:
api_key = "secret_api_key"

In [10]:
channel_ids = [
    "UC8uYStXS2ElBFLZVfuYzIxg"
]

In [11]:
api_service_name = "youtube"
api_version = "v3"

youtube = build(
    api_service_name, api_version, developerKey=api_key)

In [12]:
def get_channel_stats(youtube, channel_ids):

    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    for item in response["items"]:
        data = {
            "channelName": item["snippet"]["title"],
            "subscribers": item["statistics"]["subscriberCount"],
            "views": item["statistics"]["viewCount"],
            "totatViews": item["statistics"]["videoCount"],
            "playlistId": item["contentDetails"]["relatedPlaylists"]["uploads"]
        }
        all_data.append(data)
    return(pd.DataFrame(all_data))

In [13]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [14]:
channel_stats

,channelName,subscribers,views,totatViews,playlistId
0,Przez Świat Na Fazie,539000,104592562,230,UU8uYStXS2ElBFLZVfuYzIxg


In [15]:
playlist_id = "UU8uYStXS2ElBFLZVfuYzIxg"

In [16]:
def get_video_ids(youtube, playlist_id):

    video_ids = []

    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults=50
    )
    response = request.execute()

    for item in response["items"]:
        video_ids.append(item["contentDetails"]["videoId"])

    next_page_token = response.get("nextPageToken")

    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken = next_page_token
        )
        response = request.execute()

        for item in response["items"]:
            video_ids.append(item["contentDetails"]["videoId"])

        next_page_token = response.get("nextPageToken")

    return video_ids

In [17]:
video_ids = get_video_ids(youtube, playlist_id)

In [18]:
len(video_ids)

232

In [19]:
def get_video_details(youtube, video_ids):
    
    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response["items"]:
            stats = {"snippet": ["channelTitle", "title", "description", "tags", "publishedAt"],
                     "statistics": ["viewCount", "likeCount", "favouriteCount", "commentCount"],
                     "contentDetails": ["duration", "definition", "caption"]
                    }
            video_info = {}
            video_info["video_id"] = video["id"]

            for key in stats.keys():
                for value in stats[key]:
                    try:
                        video_info[value] = video[key][value]
                    except:
                        video_info[value] = None

            all_video_info.append(video_info)
        
    return pd.DataFrame(all_video_info)

In [20]:
video_df = get_video_details(youtube, video_ids)

In [21]:
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,e8EeMoFA5x0,Przez Świat Na Fazie,#167 Przez Świat na Fazie - Wypadek | UZBEKIST...,Witam Was Fazowicze i Fazowiczki! Przed Wami o...,"[przezswiatnafazie, przez, świat, uzbekistan, ...",2023-06-18T09:57:13Z,101817,6139,None,984,PT18M52S,hd,false
1,wDQ7h8DU44M,Przez Świat Na Fazie,#166 Przez Świat na Fazie - Misja specjalna | ...,Fazowicze i Fazowiczki! Przed Wami kolejny odc...,"[przezswiatnafazie, patec, patecki, patecwaria...",2023-06-11T09:57:44Z,190925,6488,None,484,PT25M5S,hd,false
2,0nSR4DXhL-g,Przez Świat Na Fazie,#165 Przez Świat na Fazie - Awantura w autobus...,Fazowicze i Fazowiczki! Przed Wami kolejny odc...,"[przezswiatnafazie, patecki, patecwariatec, fa...",2023-06-04T08:58:58Z,207214,7082,None,565,PT24M6S,hd,false
3,DZ6W6flhUfg,Przez Świat Na Fazie,#164 Przez Świat na Fazie - Przygodę czas zacz...,Witam Was serdecznie Fazowicze i Fazowiczki. P...,"[przezswiatnafazie, na fazie, przez, izrael, p...",2023-05-28T09:58:59Z,191803,8506,None,465,PT21M47S,hd,false
4,ViwsE2S0-E4,Przez Świat Na Fazie,#163 Przez Świat na Fazie - Ostry poślizg | IZ...,Fazowicze i Fazowiczki! Zapraszam Was na ostat...,"[przezswiatnafazie, na fazie, przez, izrael, p...",2023-05-21T09:57:43Z,163845,6620,None,575,PT22M59S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,NEKVy_34HVs,Przez Świat Na Fazie,5# Przez Świat na Fazie - Jachtostop,,"[przez świat na fazie, fazowski, na fazie prze...",2014-11-02T15:16:57Z,309587,3472,None,94,PT5M32S,hd,false
228,p79FR9uA0uA,Przez Świat Na Fazie,#2 Przez Świat na Fazie - Hiszpania,,"[przez świat na fazie, fazowski, na fazie prze...",2014-10-23T12:33:14Z,520501,5563,None,189,PT6M44S,hd,false
229,bLoWNzsbYug,Przez Świat Na Fazie,#4 Przez Świat na Fazie - No to płyniemy po oc...,,"[przez świat na fazie, fazowski, na fazie prze...",2014-10-10T19:57:59Z,425176,4259,None,94,PT3M34S,hd,false
230,OMjmmD9WF8Q,Przez Świat Na Fazie,#3 Przez Świat na Fazie - Giblartar,,"[przez świat na fazie, fazowski, na fazie prze...",2014-09-30T11:39:49Z,321621,2980,None,67,PT1M57S,hd,false
